<a href="https://colab.research.google.com/github/beenahmed/Parallelization-of-Neural-Networks/blob/main/finetuning_programmingdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import os
from datasets import load_dataset, Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import pandas as pd
!pip install evaluate
import evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
# Load dataset from Kaggle's input directory
data = pd.read_csv('/kaggle/input/codeforces-competitive-programming-dataset/CompetitiveProgrammingDataset.csv')
data.head()


,Unnamed: 0,problem_link,problem_name,problem_statement,input,output,editorial,coding_solution
0,0,https://codeforces.com//contest/1741/problem/A,A. Compare T-Shirt Sizes,Two T-shirt sizes are given: and . The T-shir...,The first line of the input contains a single ...,"For each test case, print on a separate line t...","Let , are the last characters of lines and ...",#include <bits/stdc++.h>\n \nusing namespace s...
1,1,https://codeforces.com//contest/1741/problem/B,B. Funny Permutation,A sequence of numbers is called if it contai...,The first line of input data contains a single...,"For each test case, print on a separate line: ...","We cannot make a funny permutation only when ,...","#include ""bits/stdc++.h""\nusing namespace std;..."
2,2,https://codeforces.com//contest/1741/problem/C,C. Minimize the Thickness,You are given a sequence consisting of inte...,The first line contains a single integer () —...,"For each test case, output one integer — the m...",Let's iterate over the length of the first seg...,#include <bits/stdc++.h>\n\nusing namespace st...
3,3,https://codeforces.com//contest/1741/problem/D,D. Masha and a Beautiful Tree,The girl named Masha was walking in the forest...,The first line contains single integer () — n...,"For each test case in a separate line, print t...",Let some vertex be responsible for a segment o...,#include <bits/stdc++.h>\n\nusing namespace st...
4,4,https://codeforces.com//contest/1741/problem/E,E. Sending a Sequence Over the Network,The sequence is sent over the network as foll...,The first line of input data contains a single...,For each test case print on a separate line: ...,Let's introduce the dynamics. if on the prefi...,#include <bits/stdc++.h>\n\nusing namespace st...


In [ ]:
# Assuming the columns are named 'problem_statement', 'coding_solution', and 'input' as per your description
# Select relevant columns
data = data[['problem_statement', 'coding_solution', 'input']]

# Concatenate problem_statement, input, and coding_solution into a single column for language modeling
data['text'] = data['problem_statement'].astype(str) + "\nInput: " + data['input'].astype(str) + "\nSolution:\n" + data['coding_solution'].astype(str)

# Convert DataFrame to Dataset object
dataset = Dataset.from_pandas(data[['text']])

# Split the dataset into train and test
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']


In [ ]:
# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set pad token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Apply tokenization with parallelization
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"], num_proc=4)
tokenized_test = test_dataset.map(tokenize_function, batched=True, remove_columns=["text"], num_proc=4)

# Add labels for language modeling
tokenized_train = tokenized_train.map(lambda examples: {'labels': examples['input_ids']}, batched=True, num_proc=4)
tokenized_test = tokenized_test.map(lambda examples: {'labels': examples['input_ids']}, batched=True, num_proc=4)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map (num_proc=4):   0%|          | 0/1395 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/155 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1395 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/155 [00:00<?, ? examples/s]

In [ ]:
# Initialize model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    # Enable multi-GPU training
    fp16=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# Fine-tune the model
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.994315
2,1.340100,0.949962


TrainOutput(global_step=698, training_loss=1.2618364044451782, metrics={'train_runtime': 643.0198, 'train_samples_per_second': 4.339, 'train_steps_per_second': 1.086, 'total_flos': 1458009538560000.0, 'train_loss': 1.2618364044451782, 'epoch': 2.0})

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(results)


{'eval_loss': 0.94996178150177, 'eval_runtime': 11.2232, 'eval_samples_per_second': 13.811, 'eval_steps_per_second': 3.475, 'epoch': 2.0}


In [ ]:
# Save the model and tokenizer
model.save_pretrained('/kaggle/working/fine-tuned-gpt2')
tokenizer.save_pretrained('/kaggle/working/fine-tuned-gpt2')

('/kaggle/working/fine-tuned-gpt2/tokenizer_config.json',
 '/kaggle/working/fine-tuned-gpt2/special_tokens_map.json',
 '/kaggle/working/fine-tuned-gpt2/vocab.json',
 '/kaggle/working/fine-tuned-gpt2/merges.txt',
 '/kaggle/working/fine-tuned-gpt2/added_tokens.json')

**Testing**

In [ ]:
# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained('/kaggle/working/fine-tuned-gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('/kaggle/working/fine-tuned-gpt2')

# Ensure the padding token is set
tokenizer.pad_token = tokenizer.eos_token

# Example problem statement
problem_statement = 'How to reverse an array'
# Encode the input text
input_ids = tokenizer.encode(problem_statement, return_tensors='pt')

# Create attention mask
attention_mask = (input_ids != tokenizer.pad_token_id).long()

# Generate text using the fine-tuned model
output = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=1000,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    pad_token_id=tokenizer.eos_token_id,
    num_beams=5,
    early_stopping=True
)

# Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("Problem Statement: ", problem_statement)
print("Generated Solution: ", generated_text)


Problem Statement:  How to reverse an array
Generated Solution:  How to reverse an array?
Input: The first line of the input contains one integer  () — the number of test cases. The second line contains  integers  (), where  is the array's length.
Solution:
#include <bits/stdc++.h>
using namespace std;

int main() {
	freopen("input.txt", "r", stdin); cin.tie(0, stdout); cout << "\n";
}
